# Cross Entropy

交差エントロピー

$$
E(x,y) = - \sum_{k} y_k \log{x_k}
$$

In [1]:
import torch
from torch import nn


---

## `nn.CrossEntropyLoss`

- [CrossEntropyLoss — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)

<br>

PyTorchで用意されているclass。  
ここにはsoftmax関数も含まれているので、これを使う場合はモデルにsoftmaxを入れる必要がない。

$$
\begin{align}
y
    &= f(x, t) \\
    &= -\sum_k t_k \log (softmax(x_k))
\end{align}
$$

- $f(x, t)$: torchの交差エントロピー
- $x$: 入力
- $t$: 正解ラベル

In [2]:
ce = nn.CrossEntropyLoss()

In [3]:
x = torch.tensor([0, 1], dtype=torch.float32)
l = torch.tensor([1, 0], dtype=torch.float32)
y = ce(x, l)
y.item()

1.31326162815094

softmaxが勝手に入るので、二つのone-hotベクトルが一致していても出力が0にならない。

In [4]:
x = torch.tensor([1, 0], dtype=torch.float32)
l = torch.tensor([1, 0], dtype=torch.float32)
y = ce(x, l)
y.item()

0.31326165795326233

`float`以外は入力できない。

In [5]:
x = torch.tensor([1, 0])
l = torch.tensor([1, 0])
print("dtypes:", x.dtype, l.dtype)
try:
    ce(x, l)
except Exception as e:
    print(e)

dtypes: torch.int64 torch.int64
Expected floating point type for target with class probabilities, got Long



---

## `Reduction`

ベクトルが複数ある場合(2次元以上のテンソル)にどのような演算を行うかを指定する引数。  

- `none`: 各ベクトルの損失を計算する
- `mean`: 全ベクトルの損失の平均を計算する
- `sum`: 全ベクトルの損失の合計を計算する

デフォルト: `mean`

In [6]:
# none
ce = nn.CrossEntropyLoss(reduction='none')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([[1, 0], [0, 1]], dtype=torch.float32)
y = ce(x, l)
y

tensor([0.3133, 0.6931])

In [7]:
# mean
ce = nn.CrossEntropyLoss(reduction='mean')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([[1, 0], [0, 1]], dtype=torch.float32)
y = ce(x, l)
y

tensor(0.5032)

In [8]:
# sum
ce = nn.CrossEntropyLoss(reduction='sum')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([[1, 0], [0, 1]], dtype=torch.float32)
y = ce(x, l)
y

tensor(1.0064)


---

## クラスラベルの直接入力

クラスラベルはone-hotベクトルでなく直接整数値を入力することもできる。

In [9]:
ce = nn.CrossEntropyLoss()
x = torch.tensor([[1, 0, 0], [0.5, 0.3, 0.2]], dtype=torch.float32)
l = torch.tensor([0, 2])
y = ce(x, l)
y

tensor(0.8956)


---

## `ignore_index`

演算時に無視するクラスラベルを指定する引数。

In [10]:
# 何も指定しない場合
ce = nn.CrossEntropyLoss(reduction='none')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([0, 1])
y = ce(x, l)
y

tensor([0.3133, 0.6931])

1つ目のベクトルはクラス0, 2つ目はクラス1。それぞれの損失は上記。  
`ignore_index`を指定して、片方のクラスを無視してみる。  

In [11]:
ce = nn.CrossEntropyLoss(ignore_index=1, reduction='none')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([0, 1])
y = ce(x, l)
y

tensor([0.3133, 0.0000])

クラス1を無視するように指定した結果、対応する損失が0になった。

`reduction`を`'mean'`や`'sum'`にしてもちゃんと無視した状態で値が計算される。

In [12]:
ce = nn.CrossEntropyLoss(ignore_index=1, reduction='mean')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([0, 1])
y = ce(x, l)
y

tensor(0.3133)

またこの引数を使用する場合はクラスラベルを直接入力する必要がある。

In [13]:
ce = nn.CrossEntropyLoss(ignore_index=1, reduction='none')
x = torch.tensor([[1, 0], [0.5, 0.5]], dtype=torch.float32)
l = torch.tensor([[0, 1], [0, 1]], dtype=torch.float32)
try:
    ce(x, l)
except Exception as e:
    print(e)

ignore_index is not supported for floating point target



---

## 形状

入力tensorの形状。対応しているものは以下。

- $(C)$
- $(N, C)$
- $(N, C, d_1, d_2, ..., d_K)$

### 1次元

$(C)$

In [14]:
ce = nn.CrossEntropyLoss()
x = torch.tensor([0, 1], dtype=torch.float32)
l = torch.tensor([1, 0], dtype=torch.float32)
y = ce(x, l)
y

tensor(1.3133)

### 2次元

$(N, C)$  
`reduction`に依って挙動が変わる。

In [15]:
ce = nn.CrossEntropyLoss()
x = torch.tensor([0, 1], dtype=torch.float32).reshape(1, -1)
l = torch.tensor([1, 0], dtype=torch.float32).reshape(1, -1)
y = ce(x, l)
y

tensor(1.3133)

### 3次元以上

$(N, C, d1, d2, \cdots, d_K)$

最後の軸をクラス数として扱ってくれる訳ではないので注意。

In [16]:
ce = nn.CrossEntropyLoss()
x = torch.tensor([0, 1], dtype=torch.float32).reshape(1, 1, -1)
l = torch.tensor([1, 0], dtype=torch.float32).reshape(1, 1, -1)
y = ce(x, l)
y

tensor(-0.)

これはクラス数1と解釈されているので0になる（1次元ベクトルをsoftmaxにかけると必ず1になり、$\log1=0$であるため、$\frac{(0 + 0)}{2}=0$）。  
演算は以下と同じということになる。

In [17]:
ce = nn.CrossEntropyLoss()
zero = torch.tensor([0], dtype=torch.float32)
one = torch.tensor([1], dtype=torch.float32)
y = (ce(zero, one) + ce(one, zero)).mean()
y

tensor(0.)